Forecast Data, https://open-meteo.com/en/docs/knmi-api
-7 to +7 days (Past days + Predictions), as historical weather API has a 5 days delay...
Weather features included:


In [6]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime, timedelta
import time

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define locations
locations = [
    ("Eelde", 53.214469, 6.566481),
    ("DeBilt", 52.108499, 5.180616),
    ("Vlissingen", 51.455677, 3.576488),
    ("Sittard", 50.998299, 5.86291)
]

# Make sure all required weather variables are listed here
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": [lat for _, lat, _ in locations],
    "longitude": [lon for _, _, lon in locations],
    "hourly": [
        "temperature_2m", "cloud_cover", "wind_speed_10m", 
        "diffuse_radiation", "direct_normal_irradiance", "shortwave_radiation", "apparent_temperature"],
    "models": "knmi_seamless",
    "past_days": 7
}

while True:
    fetch_timestamp = datetime.utcnow()
    responses = openmeteo.weather_api(url, params=params)

    # Collect data for all locations
    dataframes = []
    for idx, (location_name, _, _) in enumerate(locations):
        response = responses[idx]
        hourly = response.Hourly()
        hourly_data = {
            "date": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            ),
            "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
            "cloud_cover": hourly.Variables(1).ValuesAsNumpy(),
            "wind_speed_10m": hourly.Variables(2).ValuesAsNumpy(),
            "diffuse_radiation": hourly.Variables(3).ValuesAsNumpy(),
            "direct_normal_irradiance": hourly.Variables(4).ValuesAsNumpy(),
            "shortwave_radiation": hourly.Variables(5).ValuesAsNumpy(),
            "apparent_temperature": hourly.Variables(6).ValuesAsNumpy(),
            "location": location_name,
            "fetch_timestamp_utc": fetch_timestamp
        }
        dataframes.append(pd.DataFrame(data=hourly_data))

    # Combine all data into a single DataFrame
    final_dataframe = pd.concat(dataframes, ignore_index=True)

    # Save to CSV with timestamped filename in the Data folder
    timestamp_str = fetch_timestamp.isoformat(timespec='minutes')
    csv_filename = f"../Data/pred_weather_data_{timestamp_str}.csv"
    final_dataframe.to_csv(csv_filename, index=False)

    print(f"Data saved to {csv_filename}")

    # Wait for 3 hour before fetching again
    time.sleep(3*3600)


/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_31763/536470782.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  fetch_timestamp = datetime.utcnow()


Data saved to ../Data/pred_weather_data_2025-03-24T20:48.csv
Data saved to ../Data/pred_weather_data_2025-03-24T23:48.csv
Data saved to ../Data/pred_weather_data_2025-03-25T02:48.csv
Data saved to ../Data/pred_weather_data_2025-03-25T05:48.csv
Data saved to ../Data/pred_weather_data_2025-03-25T09:24.csv
Data saved to ../Data/pred_weather_data_2025-03-25T12:24.csv
Data saved to ../Data/pred_weather_data_2025-03-25T15:24.csv


KeyboardInterrupt: 

In [ ]:
'''#Code running on Home Assistant's Add-on Terminal: 

import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime, timedelta
import time
import os

# Setup Open-Meteo API client with cache and retry
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define locations
locations = [
    ("Eelde", 53.214469, 6.566481),
    ("DeBilt", 52.108499, 5.180616),
    ("Vlissingen", 51.455677, 3.576488),
    ("Sittard", 50.998299, 5.86291)
]

# Create data folder if not exists
data_folder = "weather_data"
os.makedirs(data_folder, exist_ok=True)

# Define API parameters
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": [lat for _, lat, _ in locations],
    "longitude": [lon for _, _, lon in locations],
    "hourly": [
        "temperature_2m", "cloud_cover", "wind_speed_10m", 
        "diffuse_radiation", "direct_normal_irradiance", "shortwave_radiation", "apparent_temperature"],
    "models": "knmi_seamless",
    "past_days": 7
}

def fetch_and_save():
    """Fetch data from API and save to CSV."""
    fetch_timestamp = datetime.utcnow()
    print(f"[{fetch_timestamp.isoformat()}] Fetching weather data...")

    try:
        responses = openmeteo.weather_api(url, params=params)
    except Exception as e:
        print(f"[ERROR] Failed to fetch data: {e}")
        return  # Skip this round and try again later

    dataframes = []
    for idx, (location_name, _, _) in enumerate(locations):
        response = responses[idx]
        hourly = response.Hourly()
        hourly_data = {
            "date": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            ),
            "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
            "cloud_cover": hourly.Variables(1).ValuesAsNumpy(),
            "wind_speed_10m": hourly.Variables(2).ValuesAsNumpy(),
            "diffuse_radiation": hourly.Variables(3).ValuesAsNumpy(),
            "direct_normal_irradiance": hourly.Variables(4).ValuesAsNumpy(),
            "shortwave_radiation": hourly.Variables(5).ValuesAsNumpy(),
            "apparent_temperature": hourly.Variables(6).ValuesAsNumpy(),
            "location": location_name,
            "fetch_timestamp_utc": fetch_timestamp
        }
        dataframes.append(pd.DataFrame(data=hourly_data))

    # Combine all location data
    final_dataframe = pd.concat(dataframes, ignore_index=True)

    # Save to CSV with timestamped filename
    timestamp_str = fetch_timestamp.strftime("%Y%m%d_%H%M")
    csv_filename = os.path.join(data_folder, f"weather_data_{timestamp_str}.csv")
    final_dataframe.to_csv(csv_filename, index=False)

    print(f"[{fetch_timestamp.isoformat()}] Data saved to {csv_filename}")

def wait_until_next_hour():
    """Wait until the start of the next hour to fetch data."""
    now = datetime.utcnow()
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
    wait_seconds = (next_hour - now).total_seconds()
    print(f"Sleeping for {int(wait_seconds)} seconds until next hour...\n")
    time.sleep(wait_seconds)

# Run hourly loop
while True:
    fetch_and_save()
    wait_until_next_hour()

/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_16619/2302601785.py:42: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  fetch_timestamp = datetime.utcnow()


[2025-03-18T09:00:14.279252] Fetching weather data...
[2025-03-18T09:00:14.279252] Data saved to weather_data/weather_data_20250318_0900.csv
Sleeping for 3585 seconds until next hour...



/var/folders/6_/0hk4r2517rd0g1wg9893jqz40000gp/T/ipykernel_16619/2302601785.py:86: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


[2025-03-18T10:00:00.020944] Fetching weather data...
[2025-03-18T10:00:00.020944] Data saved to weather_data/weather_data_20250318_1000.csv
Sleeping for 3599 seconds until next hour...

[2025-03-18T11:00:00.003213] Fetching weather data...
[2025-03-18T11:00:00.003213] Data saved to weather_data/weather_data_20250318_1100.csv
Sleeping for 3599 seconds until next hour...

[2025-03-18T11:59:59.960052] Fetching weather data...
[2025-03-18T11:59:59.960052] Data saved to weather_data/weather_data_20250318_1159.csv
Sleeping for 3599 seconds until next hour...



KeyboardInterrupt: 